In [1]:
import os
os.getcwd()

'/home/dobri/atmrad/+/2023'

In [2]:
import sys
sys.path.append('/home/dobri/atmrad')

In [3]:
from cpu.cloudiness import Plank3D

In [4]:
base_distributions = [
    {'name': 'L2', 'alpha': 1.411, 'Dm': 4.026, 'dm': 0.02286, 'eta': 0.93, 'beta': 0.3, 'cl_bottom': 1.2192},
    {'name': 'L3', 'alpha': 1.485, 'Dm': 4.020, 'dm': 0.03048, 'eta': 0.76, 'beta': -0.3, 'cl_bottom': 1.3716},
    {'name': 'T7', 'alpha': 1.35, 'Dm': 3.733, 'dm': 0.04572, 'eta': 1.2, 'beta': 0.0, 'cl_bottom': 1.24968},
    {'name': 'T6', 'alpha': 1.398, 'Dm': 3.376, 'dm': 0.03048, 'eta': 0.93, 'beta': -0.1, 'cl_bottom': 1.0668},
    {'name': 'T8', 'alpha': 1.485, 'Dm': 4.02, 'dm': 0.06096, 'eta': 1.2, 'beta': 0.4, 'cl_bottom': 1.3716},
    {'name': 'T9', 'alpha': 2.485, 'Dm': 2.656, 'dm': 0.04572, 'eta': 1.3, 'beta': 0.3, 'cl_bottom': 1.40208},
    
    {'name': 'L1', 'alpha': 3.853, 'Dm': 1.448, 'dm': 0.01524, 'eta': 0.98, 'beta': 0.0, 'cl_bottom': 0.54864},
    {'name': 'T5', 'alpha': 2.051, 'Dm': 2.574, 'dm': 0.02286, 'eta': 0.85, 'beta': -0.13, 'cl_bottom': 1.11252},
    {'name': 'T3', 'alpha': 2.361, 'Dm': 2.092, 'dm': 0.01524, 'eta': 0.93, 'beta': -0.1, 'cl_bottom': 0.82296},
    {'name': 'T4', 'alpha': 2.703, 'Dm': 2.094, 'dm': 0.02286, 'eta': 0.8, 'beta': 0.0, 'cl_bottom': 0.9144},
    {'name': 'T2', 'alpha': 4.412, 'Dm': 1.126, 'dm': 0.01524, 'eta': 0.97, 'beta': 0.0, 'cl_bottom': 0.70104},
    {'name': 'T1', 'alpha': 9.07, 'Dm': 0.80485, 'dm': 0.01524, 'eta': 0.89, 'beta': 0.0, 'cl_bottom': 0.67056},
]

In [5]:
import numpy as np

distributions = []
for base_distr in base_distributions:
    distributions.append(base_distr)
    for i, beta in enumerate(np.arange(-0.9, 0.9, 0.1)):
        new_distr = base_distr.copy()
        new_distr['name'] += 'B' + str(i).zfill(2)
        new_distr['beta'] = np.round(beta, decimals=1)
        distributions.append(new_distr)
    for j, eta in enumerate(np.arange(0.53, 1.83, 0.1)):
        new_distr = base_distr.copy()
        new_distr['name'] += 'E' + str(j).zfill(2)
        new_distr['eta'] = np.round(eta, decimals=2)
        distributions.append(new_distr)
len(distributions)

384

In [6]:
# domain parameters
H = 20.  # высота атмосферы
d = 100  # дискретизация по высоте
X = 50  # (км) горизонтальная протяженность моделируемой атмосферной ячейки
res = 300  # горизонтальная дискретизация

In [7]:
seed = 42  # состояние генератора случайных чисел

In [ ]:
# SEQUENTIAL

import os
import dill
import datetime

percentage = np.linspace(0.2, 0.7, 20, endpoint=True)[::-1]

if not os.path.exists('HMAPS'):
    os.makedirs('HMAPS')

start_time = datetime.datetime.now()

for i, distr in enumerate(distributions):
    print('\n\nProcessing {} ...\t|\t {} left out of {}\n'.format(distr['name'], 
                                                                 len(distributions) - i, 
                                                                 len(distributions)))
    alpha, Dm, dm, eta, beta, cl_bottom = \
        distr['alpha'], distr['Dm'], distr['dm'], distr['eta'], distr['beta'], distr['cl_bottom']
    
    xi = -np.exp(-alpha * Dm) * (((alpha * Dm) ** 2) / 2 + alpha * Dm + 1) + \
        np.exp(-alpha * dm) * (((alpha * dm) ** 2) / 2 + alpha * dm + 1)
    print('xi\t', xi)
    
    for j, required_percentage in enumerate(percentage):
        print('\n\nRequired %: {:.2f}'.format(required_percentage * 100.))
        K = 2 * np.power(alpha, 3) * (X * X * required_percentage) / (np.pi * xi)
        print('K\t', K)
    
        p = Plank3D(kilometers=(X, X, H), nodes=(res, res, d), clouds_bottom=cl_bottom)
        print('Generating cloud distribution...')
        try:
            clouds = p.generate_clouds(
                Dm=Dm, dm=dm, K=K, alpha=alpha, beta=beta, eta=eta, seed=seed, timeout=1., verbose=True
            )
        except TimeoutError:
            print('\n ...time is over')
            continue
            
        N_analytical = K / alpha * (np.exp(-alpha * dm) - np.exp(-alpha * Dm))
        N_fact = len(clouds)
        print('N\tanalytical: {}\t\tactual: {}'.format(N_analytical, N_fact))
        
        hmap = p.height_map2d_(clouds)
        
        sky_cover = np.sum(np.pi * np.power(np.array([cloud.rx for cloud in clouds]), 2))
        cover_percentage = sky_cover / (X * X)
        cover_percentage_d = np.count_nonzero(hmap) / (res * res)
        sky_cover_d = cover_percentage_d * (X * X)
        
        print('%\tbefore digitizing: {}\t\tafter digitizing: {}'.format(
            cover_percentage * 100., cover_percentage_d * 100.))
        
        with open(os.path.join('HMAPS',
                      distr['name'] + '_P' + str(int(np.round(required_percentage * 100., decimals=0))) + '.dat'), 
                  'wb') as dat:
            dill.dump(np.asarray(hmap), dat, recurse=True)
            
        print('{:.2f}% READY'.format((j + 1) / len(percentage) * 100.))
        
        elapsed = datetime.datetime.now() - start_time
        days = elapsed.days
        hours = elapsed.seconds // 3600
        minutes = (elapsed.seconds - hours * 3600) // 60
        seconds = elapsed.seconds - hours * 3600 - minutes * 60
        print('\nElapsed: {} d\t{} h\t{} m\t{} s'.format(days, hours, minutes, seconds))

In [8]:
# PARALLEL

import os
import dill
from cpu.core.multi import *

percentage = np.linspace(0.2, 0.7, 20, endpoint=True)[::-1]

n_workers = 8

if not os.path.exists('HMAPS'):
    os.makedirs('HMAPS')

def process(distr:  dict) -> None:
    alpha, Dm, dm, eta, beta, cl_bottom = \
        distr['alpha'], distr['Dm'], distr['dm'], distr['eta'], distr['beta'], distr['cl_bottom']

    xi = -np.exp(-alpha * Dm) * (((alpha * Dm) ** 2) / 2 + alpha * Dm + 1) + \
        np.exp(-alpha * dm) * (((alpha * dm) ** 2) / 2 + alpha * dm + 1)

    for j, required_percentage in enumerate(percentage):
        K = 2 * np.power(alpha, 3) * (X * X * required_percentage) / (np.pi * xi)

        p = Plank3D(kilometers=(X, X, H), nodes=(res, res, d), clouds_bottom=cl_bottom)
        try:
            clouds = p.generate_clouds(
                Dm=Dm, dm=dm, K=K, alpha=alpha, beta=beta, eta=eta, seed=seed, timeout=1., verbose=False,
            )
        except TimeoutError:
            continue

        hmap = p.height_map2d_(clouds)

        with open(os.path.join('HMAPS',
                               distr['name'] + '_P' + str(
                                   int(np.round(required_percentage * 100., decimals=0))) + '.dat'),
                  'wb') as dat:
            dill.dump(np.asarray(hmap), dat, recurse=True)

        print('\r{}\t --- \t{:.2f}% READY'.format(distr['name'], (j + 1) / len(percentage) * 100.),
              end='    ', flush=True)
        
    
with Manager() as manager:
    processes = []
    for i, d in enumerate(distributions):
        p = Process(target=process, args=(d,))
        processes.append(p)
    do(processes, n_workers)

L2	 --- 	40.00% READYADY                                                                                                                                                                                                                    

Process Process-4:
Process Process-2:
Process Process-7:
Process Process-9:
Process Process-3:
Process Process-6:
Process Process-8:
Process Process-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10

KeyboardInterrupt: 